<a href="https://colab.research.google.com/github/Chirox03/Chirox03.github.io/blob/main/Copy_of_%5BTempoRun%5D_Information_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download data

In [33]:
import json
#Download lyrics
!gdown --id 1V7WxGaVv3UamD4AM5qnqDNzkBSw6paQy
#Download queries
!gdown --id 1srNAjKMTncZUgxDTZZHfMnTbPd9ic2xx

Downloading...
From: https://drive.google.com/uc?id=1V7WxGaVv3UamD4AM5qnqDNzkBSw6paQy
To: /content/2335_Songs.json
100% 4.33M/4.33M [00:00<00:00, 164MB/s]
Downloading...
From: https://drive.google.com/uc?id=1srNAjKMTncZUgxDTZZHfMnTbPd9ic2xx
To: /content/500_queries.json
100% 39.0k/39.0k [00:00<00:00, 47.7MB/s]


# Preliminaries & Corpus Creation

In [ ]:
# Install required libraries
!pip install rank-bm25

In [ ]:
!pip install -q tf-models-official==2.3.0

     |████████████████████████████████| 840 kB 12.8 MB/s 
     |████████████████████████████████| 99 kB 9.0 MB/s 
     |████████████████████████████████| 352 kB 43.5 MB/s 
     |████████████████████████████████| 234 kB 47.3 MB/s 
     |████████████████████████████████| 47.8 MB 78 kB/s 
     |████████████████████████████████| 1.2 MB 34.2 MB/s 
     |████████████████████████████████| 1.1 MB 42.4 MB/s 
     |████████████████████████████████| 462 kB 47.1 MB/s 


In [ ]:
# Import required libraries
import os
import pprint as pp
import numpy as np
import json
import tensorflow as tf 
from gensim.corpora import WikiCorpus
from rank_bm25 import BM25Okapi, BM25Plus

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tqdm import tqdm
# link to data
CORPUS_SONG_JSON = "2335_Songs.json"
QUERIES_JSON = "500_queries.json"

In [ ]:
with open(CORPUS_SONG_JSON, 'r', encoding='utf-8') as fo:
  corpus_data = json.load(fo)

with open(QUERIES_JSON, 'r', encoding='utf-8') as fo:
  queries_test = json.load(fo)

In [ ]:
corpus_data

[{'author': 'Trần Bình',
  'lyrics': 'Bài hát: Tình Yêu Thuỷ Chung - Trần Bình, Việt Linh\nCó phải là duyên khi anh gặp em\nTừ kiếp trước, thề ước chung đôi\nCùng hân hoan\nNhìn nhau đắm đuối\nLòng quyến luyến,\nChuyện mãi không thôi\nEm từ nơi đâu đã đến nơi này\nAnh hân hoan\nTrao Em vòng tay\nVà đôi ta,\nSống chung hạnh phúc\nCùng xây đắp,\nCho mối duyên này\nEm ơi Em ơi, dù phong ba\nDù bão tố, dù nắng cháy\nAnh vẫn yêu Em, mãi mãi không thôi\nAnh ơi Anh ơi, dù gian nan\nDù khốn khó Ta vẫn yêu nhau,\nĐến cuối cuộc đời\nTa vẫn bên nhau qua bao buồn vui\nTa yêu nhau, tình yêu đắm say\nTình yêu ta đẹp, tình yêu thắm thiết\nHạnh phúc trăm năm\nTình yêu thuỷ chung\n \n',
  'song_id': '0000',
  'title': 'Tình Yêu Thuỷ Chung'},
 {'author': 'Trần Bình',
  'lyrics': 'Bài hát: Lá Thư Của Cha - Trần Bình\nTừ ngày con ra đi\nCha lòng buồn thương nhớ\nCô đơn và sầu muộn\nBao ngày tháng mong chờ\nTừ ngày con ra đi\nTới thời gian bây giờ\nThấm thoát mười hai năm chẵn\nCha thường đứng ngồi ngẩn ng

In [ ]:
import re
import string
!pip3 install vncorenlp

     |████████████████████████████████| 2.6 MB 12.7 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=8a07c122b386a636c15bf4abb00291408641e8ab78b14a0b503b9cf7750ebc6b
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp


#remove lower case, punctuation, stopwword


In [ ]:
!pip install https://gitlab.com/trungtv/vi_spacy/-/raw/master/vi_core_news_lg/dist/vi_core_news_lg-0.0.1.tar.gz


In [ ]:
import unidecode

In [ ]:
def remove_accent(text):
    return unidecode.unidecode(text)

In [ ]:
def get_stopwords_list(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))


In [40]:

stopwords_path = "/content/drive/MyDrive/Stopword_vietnamese_data/vietnamese-stopwords.txt"
stopwords = get_stopwords_list(stopwords_path)

In [42]:
stopwords[0:10]

['tuy thế',
 'duy chỉ',
 'mỗi lúc',
 'tột cùng',
 'nghe đâu',
 'ngay từ',
 'bất thình lình',
 'đồng thời',
 'ba họ',
 'xuất hiện']

In [44]:
def remove_stop_words(corpus):
    results = []
    for text in corpus:
        tmp = text.split(' ')
        for stop_word in stopwords:
            if stop_word in tmp:
                tmp.remove(stopword)
        results.append(" ".join(tmp))
        
    return results

In [50]:
def normalize_text(lyric):
      lyric = lyric.lower()
      lyric = lyric.replace('\n', ' ')
      lyric = lyric.translate({ord('\n'): None})
      lyric = lyric.translate(str.maketrans("","",string.punctuation))
      lyric = re.sub('[dk]', ' ',lyric) 
      lyric = re.sub('verse', ' ',lyric)
      lyric = re.sub('vhorus', ' ',lyric)
      lyric = re.sub('ver', ' ',lyric)
      lyric = re.sub('1', 'một',lyric)
      lyric = remove_accent(lyric)
      lyric = remove_stop_words(lyric)

      return lyric

In [48]:
def read_corpus(corpus_data):
    """
    Function that iteratively reads the saved articles from the corpus directory
    and appends the text to a list.
    
    Parameters
    ----------
    @param corpus_directory: str,
        A valid string specifying the path to the local directory in which the 
        files were saved using make_corpus().
        
    Returns
    -------
    @return corpus, list
        A list containing the text of an article in each element.
    """
    
    # Initialize empty corpus list
    corpus = []
    
    # For each file in the corpus directory, do...
    print("Reading local corpus, please wait...")

    cnt = 0
    for song in corpus_data:
      if song["lyrics"].split("\n")[0].startswith("Bài hát: "):
          lyric = "\n".join(song["lyrics"].split("\n")[1:])
      else:
          lyric = song["lyrics"]
      lyric = normalize_text(lyric)
      corpus.append([song["song_id"], lyric])
        
    # Done, return
    print("Done!")
    return corpus

In [51]:
# Read corpus! 
corpus = read_corpus(corpus_data)

# Look at some example...



# For some reason (only in Google Colab), this cell might need to be stopped
# for the first run, then run again. 

Reading local corpus, please wait...
Done!


In [56]:
corpus[5]

['0005',
 'mua xuan mang den niem vui trong  hap dat troi  mua dong bang gia lang di  hong tieng gia tu  bat ngan gio xa mot mau nang som mung nang xuan chao mua nang tuoi mung nang xuan chao mua nang tuoi bay chim dang hot bai ca xuan den voi doi ngan mai tuoi tham nu xinh nhu cat tieng cuoi  ngan loi chuc xuan ron rang hat vang  va loi hat ai lam long dam say    nguoi yeu hoi  hi mua xuan den la bao nhung mong cho e ap loi chim hot noi thay cho long tieng yeu thuong nay cung con gio xuan nong dem toi tinh yeu men chan hoa muon loi mua xuan hat tieng yeu cho minh dam say an tinh  ngoai  ia chim hot mung xuan hoa la chuc mung mung nghe con gio mua dong da mai xa roi cuoc doi da qua mot thoi da xa tim thay dau ngay  ai da qua tim thay dau ngay  ai da qua tinh yeu hay den cung nhau chung buoc loi tinh mua xuan dang hat cung ta cau hat chuc mung ngan loi chuc xuan ron rang hat vang va loi hat ai lam long dam say   ']

# BM25 Implementation

In [57]:
!pip install underthesea
!pip install pyvi

     |████████████████████████████████| 7.6 MB 9.5 MB/s 
     |████████████████████████████████| 581 kB 37.4 MB/s 


In [58]:
# Tokenize the corpus
from underthesea import word_tokenize

tokenized_corpus = []
for doc in tqdm(corpus):
  tokenize_lyric = word_tokenize(doc[-1]) 
  tokenize_lyric.append(f"-->{doc[0]}")
  tokenized_corpus.append(tokenize_lyric)

100%|██████████| 2335/2335 [00:33<00:00, 70.59it/s]


In [59]:
tokenized_corpus[2]

['doan',
 'buon',
 'cho',
 'toi',
 'biet',
 'may',
 'nguoi',
 'tinh',
 'roi',
 'xa',
 'toi',
 'huat',
 'bong',
 'biet',
 'co',
 'don',
 'ang',
 'ngap',
 'noi',
 'sau',
 'roi',
 'tung',
 'chieu',
 'may',
 'den',
 'tim',
 'tim',
 'mot',
 'minh',
 'buon',
 'toi',
 'buoc',
 'don',
 'coi',
 'mang',
 'trong',
 'tim',
 'noi',
 'nho',
 'mot',
 'minh',
 'cho',
 'troi',
 'chieu',
 'buon',
 'menh',
 'mang',
 'lang thang',
 'minh',
 'toi',
 'buoc',
 'tren',
 'noi',
 'sau',
 'di dau',
 've',
 'dau',
 'muon',
 'loi',
 'vo',
 'dinh',
 'doan',
 'buon',
 'biet',
 'may',
 'trong',
 'toi',
 'nho',
 'nguoi',
 'hong',
 'nho',
 'den',
 'toi',
 'chieu',
 'tim',
 'toi',
 'goi',
 'ten',
 'ai',
 'doan',
 'buon',
 'cho',
 'toi',
 'biet',
 'may',
 'hen',
 'ho',
 'em',
 'da',
 'ngo',
 'lo',
 'chieu',
 'nay',
 'toi',
 'dung',
 'mot',
 'minh',
 'ong',
 'nguoi',
 'dua',
 'chen',
 'hap',
 'chon',
 'nhin',
 'hoai',
 'hong',
 'thay',
 'em',
 'sang',
 'cho',
 'toi',
 'doan',
 'buon',
 'tinh',
 'nhan',
 '-->0002']

In [60]:
!pip install pyvi

In [61]:
# Instantiate BM25 object from the tokenized corpus
bm25 = BM25Okapi(tokenized_corpus)

In [62]:
#hàm a nhận một truy vấn chuỗi và đánh số `n` kết quả bắt buộc, 
#và trả về kết quả có liên quan nhất` n` từ kho dữ liệu của chúng ta:
def bm25okapi_search(tokenized_query, bm25, corpus, n_results = 1):
    """
    Function that takes a tokenized query and prints the first 100 words of the 
    n_results most relevant results found in the corpus, based on the BM25
    method.
    
    Parameters
    ----------
    @param tokenized_query: list, array-like
        A valid list containing the tokenized query.
    @param bm25: BM25 object,
        A valid object of type BM25 (BM25Okapi or BM25Plus) from the library
        `rank-bm25`, initialized with a valid corpus.
    @param corpus: list, array-like
        A valid list containing the corpus from which the BM25 object has been 
        initialized. As returned from function read_corpus().
    @param n_results: int, default = 1
        The number of top results to print.
    """
    
    # We skip checking validity of arguments for now... We assume the user 
    # knows what they're doing.
    
    # Get top results for the query
    top_results = bm25.get_top_n(tokenized_query, corpus, n = n_results)
    
    # Take words from each result
    top_results_id = [' '.join(top_result).split("-->")[-1] 
                             for top_result in top_results]
    
    # Return results
    return top_results_id

# Process & Export file submission

In [66]:
import csv
from pyvi import ViUtils
import re

with open("submission.csv", "w") as csv_writer:
  fieldnames = ['query_id', 'song_id']
  writer = csv.DictWriter(csv_writer, fieldnames=fieldnames)
  writer.writeheader()
  for ques in tqdm(queries_test):
    preprocess_query = " ".join(ques["query"].lower().replace("/", "").strip("").split())
    preprocess_query = normalize_text(preprocess_query)
    print(preprocess_query)
    tokenized_query = word_tokenize(preprocess_query)
    results = bm25okapi_search(tokenized_query = tokenized_query, bm25 = bm25, corpus = tokenized_corpus, n_results = 10)
    writer.writerow({'query_id': ques["query_id"], "song_id": " ".join(results)})

  2%|▏         | 10/500 [00:00<00:05, 95.52it/s]

cho qua bao nam mong buon quen  au
chuyen minh gio  et thuc that roi
 uoi hien nha nhin mua roi
di ngang qua oh my la y
gac lai moi thu cua ngay hom qua 
bong  hoi lam chieu  ap  ieu may bay nhap nho
yeah yeah chung may nen nho mot dieu la vay man its true
toi nghe nhung bai lofi ma em so li e va nghe chuyen em  e
tinh cam nay da het that roi
gom gop yeu thuong que nha ang het cho nguoi tinh xa 
vi tron ngu trua o ngoai duong ao ba lo va ra nang
ngon ngon hom nay co mon ngon ngon
 u cho em co sai mai thoi cu lam lai
nghieng doi xuong nhin xuong moi tinh
pen ghe sang tha thinh roi  eo phang xiem y cang
trong anh mat hy vong
nhung an tinh ngay do co bao nhieu
thanh qua se vang hon ca tieng sam gam
thanh mua xuan ngot ngao
trai dat hoa thanh vong compa
li tri dat ben long nguc trai
cho con nguoi tri tue nhung lai lay ca di con tim


  7%|▋         | 33/500 [00:00<00:04, 94.05it/s] 

nguyen team da hi sinh
trai tim bang gia mo uoc ngay xuan
nguoi dung noi den nua dung tham mong nhu xua
hom nay se tro thanh ngay qua  hu  hi man dem tan sau anh mat troi don binh minh
oi thang nam got chan mon tren phiem  u
money money s rt
 o t qua ghe t choi qua te t choi qua nhieu thu dang ghet nhe
ta dau muon dem nay  an tan phai
chac tay sung dem ngay canh giu bien cuong
neu dinh menh da bat ta phai xa cach
anh oi anh bi   hong co nguoi yeu linh bien hon nuong song bay theo
thay em di chan em mang doi ni e
no  zo ban pha
dai  a ru di dua vi hom nay anh ranh
babe  hong can chuoc anh da say roi
em la may cung lan gio xa
tham chi neu ban   the o ben nguoi do
vi baby girl anh yeu em mat roi
chang tiec chi bao ngay ngay thang co nhau
noi tu tay sang dong
i poppe  me a xan


 11%|█         | 56/500 [00:00<00:04, 101.71it/s]

...

i just wanna say awh
tim mot  e cho em mot thu ma chi ban than em co the mang lai
i nee  you tonight
2 ban tay trang
em cu nam va goi dau
dau phai la hip hop chat an sau vao mau  hong mot ai sanh bang
thuong mot nguoi ma dau co thuong
bao nam qua  u xa em nhung toi van nho
ve noi chi co ngoi nha va hai chung ta
we runnin this  trap this 
trang roi nhoa tren mai
tam hon ta tan tanh
loo in to  a s y ohhhhh
we runnin this  trap this 
dung cho toi loi buom hoa cho loi toi muon biet
party yes o night
ta dem than lieu cho to quoc
baby dem qua la mua giong nhu dang t hay anh trao den bao dieu can noi say goo bye say goo  bye
la  o vinh quang
vang rong trat nguyen cay do
se cuon di mau
thu minh  hong can thi minh se nhanh quen
toi van dang mong cho em se buoc di ve


 98%|█████████▊| 488/500 [00:05<00:00, 108.58it/s]

ngay mua thang nang con buon
tung ngay van luon hang co bong hinh em
tao lon len voi su cuc  ho va  hi do nha  hong co giuong nam
tung  ong nguoi  ong nguoi luot qua
 hong ai biet truoc ta thinh linh
no moi nhan ra dong ho sai co the sua
con ai cho dau
ma em van chua tan lam
ta da yeu roi nguoi ay cao sang con ta  iep phong tran
all those wor s he sai 
tien ve sai gon ta tien ve thanh do
from mixi with love
niem vui bao vay day
em dau hay hanh phuc trong tay
ta thoat com me
nhieu luc  o the  iem soatshut up s ip off
 eep it real nhu young boy dang on my way yeah yeah yeah
3 trang do de tao cuoc tranh luan chinh thong
con  y niem chung ta phai di xa
ai no vo tinh danh lia xa ben mo
tim vong tay am ap tran day
goi nguoi em yeu chut tinh linh dao
de gio chon duoc vui minh vao nhung chon choi


100%|██████████| 500/500 [00:05<00:00, 88.52it/s] 

vat doi sao  oi ta  hong the trach em phu tinh ta
nhin quang tham  uoi mat em nga nau
chut thoi chut thoi nhung mai tro thanh nho nhung
baby anh  hong thich em
tu thuo xa nguoi may mua la roi
so baby lets go
va con ai nua trong cuoc doi
2h sang o  ose


In [65]:
# check queries


{'query': 'cho qua bao năm mộng buon quen dấu', 'query_id': '000'}